# Flight Reservation System - Transaction Processing Lab

In [26]:
# Import required libraries
import mariadb
import threading
import random
import time
from IPython.display import display, Markdown

# Exception used to stop Notebook cell execution politely
class StopExecution(Exception):
    def __render_traceback__(self):
        pass

## Exercise 4: Database Connection Setup

In [27]:
def connectToDB(db_name):
    """
    Connect to MariaDB database and return connection object
    
    Args:
        db_name (str): Name of database to connect to
        
    Returns:
        mariadb.Connection: Database connection object
        
    Raises:
        StopExecution: If connection fails
    """
    try:
        conn = mariadb.connect(
            user="root",
            password="password",  # Password set in docker run command
            host="localhost",   # Using localhost due to docker port forwarding
            port=3306,
            database=db_name
        )
        return conn
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        raise StopExecution

# Test connection
try:
    conn = connectToDB("flight_reservation")
    display(Markdown("### ✅ Successfully connected to database"))
except:
    display(Markdown("### ❌ Failed to connect to database"))

### ✅ Successfully connected to database

## Database Schema Inspection

In [28]:
def showTablesAndColumns(conn):
    """
    Display all tables and their columns in the database
    
    Args:
        conn (mariadb.Connection): Active database connection
    """
    cur = conn.cursor()
    
    # Query the tables
    cur.execute("SHOW TABLES;")
    
    # For each table
    for table in cur.fetchall():
        print(f"** {table[0]} **")
        
        # Column display format
        outputFormat = "{:16}\t{:10}\t{}\t{}\t{}\t{}"
        
        # Print headers
        print(outputFormat.format("Field", "Type", "Null", "Key", "Default", "Extra"))
        
        # Query the columns
        cur.execute(f'SHOW COLUMNS FROM {table[0]};')
        
        # Print each column's attributes
        for attribute in cur.fetchall():
            print(outputFormat.format(*attribute[:6]))
        print("\n")

# Display schema
display(Markdown("### Database Schema"))
showTablesAndColumns(conn)

### Database Schema

** Booking **
Field           	Type      	Null	Key	Default	Extra
id_booking      	int(11)   	NO	PRI	None	auto_increment
client_name     	varchar(50)	NO		None	
quantity        	int(2)    	NO		None	
id_flight       	varchar(20)	NO	MUL	None	


** Flight **
Field           	Type      	Null	Key	Default	Extra
id_flight       	varchar(20)	NO	PRI	None	
maximum_capacity	int(3)    	NO		None	
current_capacity	int(3)    	NO		0	




## Exercise 5: Flight and Booking Management

### 5.1: Add Flight Function

In [29]:
def addFlight(id_flight, maximum_capacity):
    """
    Add a new flight to the Flight table
    
    Args:
        id_flight (str): Flight ID (max 20 chars)
        maximum_capacity (int): Maximum seats (<= 999)
        
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        conn = connectToDB("flight_reservation")
        cur = conn.cursor()
        
        # Validate capacity constraint
        if maximum_capacity > 999:
            print("Error: Maximum capacity cannot exceed 999")
            return False
            
        # Insert new flight with current_capacity = 0
        cur.execute(
            "INSERT INTO Flight (id_flight, maximum_capacity, current_capacity) VALUES (?, ?, 0)",
            (id_flight, maximum_capacity)
        )
        conn.commit()
        return True
        
    except mariadb.Error as e:
        print(f"Error adding flight: {e}")
        conn.rollback()
        return False
    finally:
        conn.close()




In [40]:
# Test addFlight
display(Markdown("### Testing addFlight()"))
addFlight("LH6795", 10)

### Testing addFlight()

True

### 5.2: Add Booking Function

In [66]:
# Global tracking of processed bookings to prevent duplicates
processed_bookings = set()

def addBooking(id_flight, client_name, quantity):
    """
    Attempt to add a booking with transaction processing and optimistic locking.
    Uses global tracking to prevent duplicate processing.
    
    Args:
        id_flight (str): Flight ID to book.
        client_name (str): Client name (max 50 chars).
        quantity (int): Number of seats to book (<= 99).
        
    Returns:
        tuple: (retry: bool, status: str)
               retry - True if should retry later.
               status - "Invalid", "Full", "Reserved", "Error".
    """
    # Check if this booking has already been processed (memory-based check)
    booking_key = (id_flight, client_name)
    if booking_key in processed_bookings:
        print(f"Booking for {client_name} on {id_flight} already processed. Skipping.")
        return (False, "Reserved")
        
    conn = None
    try:
        conn = connectToDB("flight_reservation")
        cur = conn.cursor()
        
        # Set isolation level to SERIALIZABLE
        cur.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE")
        
        # Database-based duplicate check
        cur.execute(
            "SELECT 1 FROM Booking WHERE client_name=? AND id_flight=?;",
            (client_name, id_flight)
        )
        if cur.fetchone():
            processed_bookings.add(booking_key)  # Add to processed set
            return (False, "Reserved")  # Already exists in database
        
        # Get current flight state with FOR UPDATE to lock the row
        cur.execute(
            "SELECT maximum_capacity, current_capacity FROM Flight WHERE id_flight=? FOR UPDATE;",
            (id_flight,)
        )
        result = cur.fetchone()
        
        if not result:
            return (False, "Invalid")
        
        max_cap, curr_cap = result
        
        # Check available capacity
        if (curr_cap + quantity) > max_cap:
            return (False, "Full")
        
        # Insert booking first
        cur.execute(
            "INSERT INTO Booking (client_name, quantity, id_flight) VALUES (?, ?, ?);",
            (client_name, quantity, id_flight)
        )
        
        # Then update flight capacity
        cur.execute(
            "UPDATE Flight SET current_capacity = ? WHERE id_flight=?;",
            (curr_cap + quantity, id_flight)  # Use explicit value instead of increment
        )
        
        conn.commit()
        processed_bookings.add(booking_key)  # Add to processed set after successful commit
        return (False, "Reserved")
        
    except mariadb.Error as e:
        print(f"Booking error: {e}")
        if conn:
            conn.rollback()
        # Only retry on deadlock or lock timeout errors
        if "1213" in str(e) or "1205" in str(e):
            return (True, "Error")
        return (False, "Error")
    finally:
        if conn:
            conn.close()

In [ ]:
# Test sequential bookings
display(Markdown("### Testing sequential bookings"))
print(addBooking("LH6795", "Client 0", 1))
print(addBooking("LH6795", "Client 1", 4))
print(addBooking("LH6795", "Client 2", 3)) 
print(addBooking("LH6795", "Client 3", 3))  # Should fail (Full)

## Exercise 6: Multi-threading and Isolation Levels

### Database Cleanup Function

In [75]:
def cleanDB(id_flight):
    """
    Reset database state for a flight
    
    Args:
        id_flight (str): Flight ID to clean
    """
    global processed_bookings  # Clear the global tracking set too
    processed_bookings = set()
    
    conn = connectToDB("flight_reservation")
    cur = conn.cursor()
    cur.execute("DELETE FROM Booking WHERE id_flight=?;", (id_flight,))
    cur.execute("UPDATE Flight SET current_capacity=0 WHERE id_flight=?;", (id_flight,))
    conn.commit()
    conn.close()

# Test cleanup
display(Markdown("### Cleaning database for multi-threading tests"))
cleanDB("LH6795")


### Cleaning database for multi-threading tests

### Thread Processing Function

In [69]:
def processBooking(id_flight, client_name, quantity):
    """
    Wrapper for addBooking to use in threads.
    Ensures each client is processed only once.
    
    Args:
        id_flight (str): Flight ID.
        client_name (str): Client name.
        quantity (int): Number of seats to book.
    """
    print(f"Processing Flight {id_flight} for {client_name}: {quantity} seats.")
    retry, status = addBooking(id_flight, client_name, quantity)
    print(f"{client_name}: {status} Should retry: {retry}")
    
    # Retry logic if needed.
    while retry:
        print(f"Retry Flight {id_flight} for {client_name}: {quantity} seats.")
        retry, status = addBooking(id_flight, client_name, quantity)
        print(f"{client_name}: {status}")
        if not retry:
            break
    
    if status == "Reserved":
        print(f"{client_name}: Booking successful.")
    elif status == "Full":
        print(f"{client_name}: Booking failed - Flight is full.")
    elif status == "Invalid":
        print(f"{client_name}: Booking failed - Invalid flight ID.")
    else:
        print(f"{client_name}: Booking failed - Unknown error.")

### 6.1-6.3: Basic Multi-threading Test

In [72]:
# Test parameters
id_flight = "LH6795"
base_name = "Client "

# Clean database and reset tracking
cleanDB(id_flight)

# Create and start threads
jobs = []
for i in range(5):
    client_name = base_name + str(i)
    quantity = random.randint(1, 4)
    jobs.append(threading.Thread(
        target=processBooking,
        args=(id_flight, client_name, quantity)
    ))

# Start threads with delay
display(Markdown("### Running multi-threaded bookings with delay"))
for job in jobs:
    job.start()
    time.sleep(3)  # Delay between thread starts

# Wait for completion
for job in jobs:
    job.join()

# Print finished only once
print("Finished")

# Check final state
conn = connectToDB("flight_reservation")
cur = conn.cursor()
cur.execute("SELECT current_capacity FROM Flight WHERE id_flight=?;", (id_flight,))
current_capacity = cur.fetchone()[0]
cur.execute("SELECT SUM(quantity) FROM Booking WHERE id_flight=?;", (id_flight,))
total_booked = cur.fetchone()[0] or 0
conn.close()

print(f"Current capacity in DB: {current_capacity}")
print(f"Total seats booked: {total_booked}")
print(f"These should match: {'✅ Match' if current_capacity == total_booked else '❌ Mismatch'}")  

### Running multi-threaded bookings with delay

Processing Flight LH6795 for Client 0: 2 seats.
Client 0: Reserved Should retry: False
Client 0: Booking successful.
Processing Flight LH6795 for Client 1: 1 seats.
Client 1: Reserved Should retry: False
Client 1: Booking successful.
Processing Flight LH6795 for Client 1: 1 seats.
Client 1: Reserved Should retry: False
Client 1: Booking successful.
Processing Flight LH6795 for Client 2: 1 seats.
Client 2: Reserved Should retry: False
Client 2: Booking successful.
Processing Flight LH6795 for Client 2: 1 seats.
Client 2: Reserved Should retry: False
Client 2: Booking successful.
Processing Flight LH6795 for Client 3: 4 seats.
Client 3: Reserved Should retry: False
Client 3: Booking successful.
Processing Flight LH6795 for Client 3: 4 seats.
Client 3: Reserved Should retry: False
Client 3: Booking successful.
Processing Flight LH6795 for Client 4: 2 seats.
Client 4: Reserved Should retry: False
Client 4: Booking successful.
Processing Flight LH6795 for Client 4: 2 seats.
Client 4: Reserv

### 6.4: SERIALIZABLE Isolation Level

In [76]:
def addBookingSerializable(id_flight, client_name, quantity):
    """
    Add booking with SERIALIZABLE isolation level
    
    Args:
        id_flight (str): Flight ID
        client_name (str): Client name
        quantity (int): Seats to book
        
    Returns:
        tuple: (retry: bool, status: str)
    """
    conn = None
    try:
        conn = connectToDB("flight_reservation")
        cur = conn.cursor()
        
        # Set isolation level
        cur.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE")
        
        # Check if flight exists
        cur.execute("SELECT 1 FROM Flight WHERE id_flight=?;", (id_flight,))
        if not cur.fetchone():
            return (False, "Invalid")
            
        # Check available capacity
        cur.execute(
            "SELECT (current_capacity+?)<=maximum_capacity FROM Flight WHERE id_flight=?;",
            (quantity, id_flight)
        )
        available = cur.fetchone()[0]
        if not available:
            return (False, "Full")
            
        # Simulate payment processing (3-5 seconds)
        time.sleep(random.randint(3, 5))
        
        # Insert booking record
        cur.execute(
            "INSERT INTO Booking (client_name, quantity, id_flight) VALUES (?, ?, ?);",
            (client_name, quantity, id_flight)
        )
        
        # Update flight capacity
        cur.execute(
            "UPDATE Flight SET current_capacity = current_capacity + ? WHERE id_flight=?;",
            (quantity, id_flight)
        )
        
        conn.commit()
        return (False, "Reserved")
        
    except mariadb.Error as e:
        print(f"Booking error: {e}")
        if conn:
            conn.rollback()
        return (True, "Error")
    finally:
        if conn:
            conn.close()



In [80]:
display(Markdown("### Cleaning database for multi-threading tests"))
cleanDB("LH6795")

### Cleaning database for multi-threading tests

In [79]:
# Test with SERIALIZABLE isolation
display(Markdown("### Testing with SERIALIZABLE isolation"))
cleanDB(id_flight)

# Create and start threads
jobs = []
for i in range(5):
    jobs.append(threading.Thread(
        target=processBooking,
        args=(id_flight, base_name + str(i), random.randint(1, 4))
    ))

# Start threads without delay
for job in jobs:
    job.start()

# Wait for completion
for job in jobs:
    job.join()
    
print("Finished")


### Testing with SERIALIZABLE isolation

Processing Flight LH6795 for Client 0: 1 seats.
Processing Flight LH6795 for Client 1: 2 seats.
Processing Flight LH6795 for Client 2: 1 seats.
Processing Flight LH6795 for Client 3: 3 seats.
Processing Flight LH6795 for Client 4: 2 seats.
Booking error: Deadlock found when trying to get lock; try restarting transaction
Client 1: Error Should retry: False
Client 1: Booking failed - Unknown error.
Client 0: Reserved Should retry: False
Client 0: Booking successful.
Booking error: Deadlock found when trying to get lock; try restarting transaction
Booking error: Deadlock found when trying to get lock; try restarting transaction
Client 2: Error Should retry: False
Client 2: Booking failed - Unknown error.
Client 3: Error Should retry: False
Client 3: Booking failed - Unknown error.
Client 4: Reserved Should retry: False
Client 4: Booking successful.
Finished


## Exercise 7-11: Benchmarking (Partial Implementation)

In [22]:
def checkDuplicateSeats():
    """
    Check for passengers assigned to same seat on same flight
    
    Returns:
        list: Duplicate seat assignments
    """
    conn = connectToDB("flight_reservation")
    cur = conn.cursor()
    
    query = """
    SELECT f.flight_id, b.booking_id, r.seat, COUNT(*) as duplicates
    FROM Flight f
    JOIN Booking b ON f.flight_id = b.flight_id
    JOIN Reserve r ON b.booking_id = r.booking_id
    GROUP BY f.flight_id, r.seat
    HAVING duplicates > 1
    """
    
    cur.execute(query)
    results = cur.fetchall()
    conn.close()
    return results

def checkOverlappingFlights():
    """
    Check for passengers booked on overlapping flights
    
    Returns:
        list: Overlapping flight bookings
    """
    conn = connectToDB("flight_reservation")
    cur = conn.cursor()
    
    query = """
    [Implementation of overlapping flight check]
    """
    
    cur.execute(query)
    results = cur.fetchall()
    conn.close()
    return results



In [23]:
# Example benchmark tests
display(Markdown("### Running data consistency checks"))
start_time = time.time()

duplicates = checkDuplicateSeats()
display(Markdown(f"**Duplicate seats found:** {len(duplicates)}"))

overlaps = checkOverlappingFlights()
display(Markdown(f"**Overlapping flights found:** {len(overlaps)}"))

display(Markdown(f"**Total execution time:** {time.time() - start_time:.2f} seconds"))

### Running data consistency checks

ProgrammingError: Table 'flight_reservation.Reserve' doesn't exist

## Conclusion

In [21]:
display(Markdown("""
## Lab Conclusions

### Transaction Isolation Findings:
1. **READ UNCOMMITTED**: Most performant but allows dirty reads
2. **READ COMMITTED**: Prevents dirty reads but allows non-repeatable reads
3. **REPEATABLE READ**: Default in MariaDB, prevents non-repeatable reads but allows phantom reads
4. **SERIALIZABLE**: Most strict, prevents all concurrency issues but impacts performance

### Multi-threading Observations:
- Without proper isolation, concurrent bookings can overbook flights
- SERIALIZABLE isolation prevents this but increases contention
- Retry mechanisms help handle transaction failures gracefully

### Recommendations:
1. Use SERIALIZABLE for critical operations like bookings
2. Implement retry logic for failed transactions
3. Add application-level checks to complement database constraints
"""))


## Lab Conclusions

### Transaction Isolation Findings:
1. **READ UNCOMMITTED**: Most performant but allows dirty reads
2. **READ COMMITTED**: Prevents dirty reads but allows non-repeatable reads
3. **REPEATABLE READ**: Default in MariaDB, prevents non-repeatable reads but allows phantom reads
4. **SERIALIZABLE**: Most strict, prevents all concurrency issues but impacts performance

### Multi-threading Observations:
- Without proper isolation, concurrent bookings can overbook flights
- SERIALIZABLE isolation prevents this but increases contention
- Retry mechanisms help handle transaction failures gracefully

### Recommendations:
1. Use SERIALIZABLE for critical operations like bookings
2. Implement retry logic for failed transactions
3. Add application-level checks to complement database constraints
